## Collect data

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

In [ ]:
!mv *.csv *.zip data/

In [ ]:
!unzip data/train_images.zip -d data/train
!unzip data/test_images.zip -d data/test

In [ ]:
!rm data/*.zip

## Process data

In [ ]:
import pandas as pd
from pathlib import Path
from fastai.vision import *
%matplotlib inline

# Data locations
path = Path("data/")

In [ ]:
test = ImageList.from_csv(path, folder="test", csv_name="test.csv", suffix=".png")
train = (ImageList.from_csv(path, folder="train", csv_name="train.csv", suffix=".png")
    .split_by_rand_pct(0.2)
    .label_from_df()
    .add_test(test)
    .transform(get_transforms(flip_vert=False), size=128)
    .databunch(path=".", bs=64)
    .normalize(imagenet_stats)
        )

In [ ]:
train.show_batch(rows=3, figsize=(7, 7))

In [ ]:
from fastai.metrics import KappaScore
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
learn = cnn_learner(train, models.resnet50, metrics=kappa)

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.save("stage-1")

## Second stage

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-5, 5e-4))

In [ ]:
learn.save("stage-2a")

In [ ]:
learn.load("stage-2a")
learn.export("models/export.pkl")

## Visualise

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses, idxs = interp.top_losses()
interp.plot_top_losses(9, figsize=(7, 7))

In [ ]:
interp.plot_confusion_matrix(figsize=(5, 5), dpi=60)

## Create test predictions

In [ ]:
preds, targs = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
submission = pd.read_csv(path/"sample_submission.csv")
submission.head()

In [ ]:
submission["diagnosis"] = preds.clone().detach().numpy().argmax(1)
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

## Create Kaggle kernel for submission

First, create a dataset to upload the trained model.

In [ ]:
!mkdir datasets
!kaggle datasets init -p datasets
!cp models/export.pkl datasets

Now, change the `id` and `title` fields in the metadata file.

In [ ]:
!kaggle datasets create -p datasets

In [ ]:
# For uploading a new version in case something went wrong
!cp models/export.pkl datasets
!rm datasets/stage-2a.pth
!kaggle datasets version -m "Correct model" -p datasets -d

Next, create an inference notebook inside `kernel` directory.

In [ ]:
!mkdir kernels
!kaggle kernels init -p kernels

Edit the metadata file to include the right competition, language, kernel type, datasets, etc. Then, push the kernel. Apparently, underscores cannot be used in the ID, so use dashes everywhere!

In [ ]:
!kaggle kernels push -p kernels